In [1]:
# Importing modules
import torch as to
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch.utils.data as to_data
from torch.utils.tensorboard import SummaryWriter as sumwriter

In [2]:
# Specify hardware for ML training (GPU default)
device = "cuda" if to.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# Quickly generate list of strings for frequency numbers and ratios
def freq_name(no_freq, include_freq=True, include_ratio=True):
    """
    Creates an ordered list of string from inputted parameters:

    no_freq = (int) number of desired frequencies
    include_freq = (bool) include the individual frequencies or not (default True)
    include_ratio = (bool) include the non-trivial ratios between frequencies or not (default True)
    """
    names = []
    if include_freq:
        for i in range(no_freq):
            names.append('f'+str(i+1))
    if include_ratio:
        for i in range(no_freq):
            for j in range(i):
                names.append('f'+str(i+1)+'/f'+str(j+1))
    return names

In [4]:
# Create Pytorch dataset class for data batching during training
class GSAE_data(to_data.Dataset):
    def __init__(self, scaled_dataframe, X_names, Y_names):
        self.len = len(scaled_dataframe)
        self.X = to.from_numpy(scaled_dataframe[X_names].to_numpy().astype('float32')).to(device)
        self.Y = to.from_numpy(scaled_dataframe[Y_names].to_numpy().astype('float32')).to(device)

    def __len__(self):
        return self.len
  
    def __getitem__(self, idx):
        X_idx = self.X[idx,:]
        Y_idx = self.Y[idx,:]
        return X_idx, Y_idx

In [5]:
def activation(activ_name):
    if activ_name=='relu':
        return to.nn.ReLU()
    elif activ_name=='lrelu':
        return to.nn.LeakyReLU()
    elif activ_name=='prelu':
        return to.nn.PReLU()
    elif activ_name=='relu6':
        return to.nn.ReLU6()
    elif activ_name=='sigmoid':
        return to.nn.Sigmoid()
    elif activ_name=='tanh':
        return to.nn.Tanh()
    elif activ_name=='silu':
        return to.nn.SiLU()
    elif activ_name=='selu':
        return to.nn.SELU()
    elif activ_name=='celu':
        return to.nn.CELU()
    elif activ_name=='gelu':
        return to.nn.GELU()
    else:
        return to.nn.ReLU()

In [6]:
class GSAE_Network(to.nn.Module):
    def __init__(self, num_X, num_Y, num_Z, he_nodes, hd_nodes, hactiv_type):
        super(GSAE_Network, self).__init__()

        self.encoder = []
        self.encoder.append(to.nn.Linear(num_X, he_nodes[0]))
        self.encoder.append(activation(hactiv_type))

        for i in range(len(he_nodes)-1):
            self.encoder.append(to.nn.Linear(he_nodes[i], he_nodes[i+1]))
            self.encoder.append(activation(hactiv_type))

        self.encoder.append(to.nn.Linear(he_nodes[-1], num_Y))

        self.encoder = to.nn.Sequential(*self.encoder).to(device)
        for i in self.encoder[::2]:
            to.nn.init.xavier_uniform_(i.weight)
            to.nn.init.zeros_(i.bias)

        self.decoder = []
        self.decoder.append(to.nn.Linear(num_Y, hd_nodes[0]))
        self.decoder.append(activation(hactiv_type))

        for i in range(len(hd_nodes)-1):
            self.decoder.append(to.nn.Linear(hd_nodes[i], hd_nodes[i+1]))
            self.decoder.append(activation(hactiv_type))

        self.decoder.append(to.nn.Linear(hd_nodes[-1], num_Z))
        
        self.decoder = to.nn.Sequential(*self.decoder).to(device)
        for i in self.decoder[::2]:
            to.nn.init.xavier_uniform_(i.weight)
            to.nn.init.zeros_(i.bias)

    def forward(self, X):
        Y = self.encoder(X)
        Z = self.decoder(Y)
        return Y, Z

In [7]:
def train_epoch(
    network,num_freq_in, num_freq_out,
    train_dataloader,
    loss_function, optimizer,
    tb_writer, epoch_ind
    ):

    loss_list = []
    loss_listY = []
    loss_listX = []

    MAPE_listY = []
    MAPE_listX = []

    for i, data in enumerate(train_dataloader):
        X, Y = data

        if epoch_ind==0 and i==0:
            tb_writer.add_graph(network, X[:,0:num_freq_in], verbose=False)

        optimizer.zero_grad()
        predictY, predictX = network(X[:,0:num_freq_in])

        lossY = loss_function(predictY, Y)
        lossX = loss_function(predictX, X)
        loss = lossY + 2*lossX

        loss_listY.append(lossY.item())
        loss_listX.append(lossX.item())
        loss_list.append(loss.item())

        loss.backward()
        optimizer.step()

        X[:,0:num_freq_out] = to.exp(X[:,0:num_freq_out])
        predictX[:,0:num_freq_out] = to.exp(predictX[:,0:num_freq_out])
 
        MAPEY = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPEX = to.mean(to.abs((X - predictX) / X)*100)

        MAPE_listY.append(MAPEY.item())
        MAPE_listX.append(MAPEX.item())
    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_lossY = to.mean(to.tensor(loss_listY, device=device)).item()
    mean_lossX = to.mean(to.tensor(loss_listX, device=device)).item()

    mean_MAPEY = to.mean(to.tensor(MAPE_listY, device=device)).item()
    mean_MAPEX = to.mean(to.tensor(MAPE_listX, device=device)).item()

    return mean_loss, mean_lossY, mean_lossX, mean_MAPEY, mean_MAPEX

def valid_epoch(
    network,num_freq_in, num_freq_out,
    valid_dataloader,
    loss_function
    ):

    loss_list = []
    loss_listY = []
    loss_listX = []

    MAPE_listY = []
    MAPE_listX = []

    for i, data in enumerate(valid_dataloader):
        X, Y = data
        predictY, predictX = network(X[:,0:num_freq_in])

        lossY = loss_function(predictY, Y)
        lossX = loss_function(predictX, X)
        loss = lossY + 2*lossX

        loss_listY.append(lossY.item())
        loss_listX.append(lossX.item())
        loss_list.append(loss.item())
        
        X[:,0:num_freq_out] = to.exp(X[:,0:num_freq_out])
        predictX[:,0:num_freq_out] = to.exp(predictX[:,0:num_freq_out])
 
        MAPEY = to.mean(to.abs((Y - predictY) / Y)*100)
        MAPEX = to.mean(to.abs((X - predictX) / X)*100)

        MAPE_listY.append(MAPEY.item())
        MAPE_listX.append(MAPEX.item())
    
    mean_loss = to.mean(to.tensor(loss_list, device=device)).item()
    mean_lossY = to.mean(to.tensor(loss_listY, device=device)).item()
    mean_lossX = to.mean(to.tensor(loss_listX, device=device)).item()

    mean_MAPEY = to.mean(to.tensor(MAPE_listY, device=device)).item()
    mean_MAPEX = to.mean(to.tensor(MAPE_listX, device=device)).item()

    return mean_loss, mean_lossY, mean_lossX, mean_MAPEY, mean_MAPEX

In [8]:
def train_GSAE(
    network, num_freq_in, num_freq_out,
    train_dataloader, valid_dataloader,
    loss_function, optimizer_type,
    epochs, learn_rate
    ):

    if optimizer_type=='adam':
        optimizer = to.optim.Adam(network.parameters(), lr=learn_rate)
    else:
        optimizer = to.optim.SGD(network.parameters(), lr=learn_rate)
    
    tb_writer = sumwriter()
    
    for i in range(epochs):
        network.train(True)
        mloss, mlossY, mlossX, mMAPEY, mMAPEX = train_epoch(network, num_freq_in, num_freq_out, train_dataloader, loss_function, optimizer, tb_writer, i)

        network.eval()
        with to.no_grad():
            vmloss, vmlossY, vmlossX, vmMAPEY, vmMAPEX = valid_epoch(network, num_freq_in, num_freq_out, valid_dataloader, loss_function)

        print('-'*50)
        print('Epoch {} / {}'.format(i+1,epochs))
        print('-'*15)
        print('Average Train Loss : {}'.format(mloss))
        print('Average Validation Loss : {}'.format(vmloss))

        tb_writer.add_scalars("Batch Mean Loss",
                            {
                                'Train' : mloss,
                                'Validation' : vmloss
                            }, i+1)

        tb_writer.add_scalars("Batch MAPE - Latent Space",
                            {
                                'Train' : mMAPEY,
                                'Validation' : vmMAPEY
                            }, i+1)

        tb_writer.add_scalars("Batch MAPE - Reconstruction",
                            {
                                'Train' : mMAPEX,
                                'Validation' : vmMAPEX
                            }, i+1)

        tb_writer.add_scalars("Batch Mean Loss - Latent Space",
                            {
                                'Train' : mlossY,
                                'Validation' : vmlossY
                            }, i+1)
    
        tb_writer.add_scalars("Batch Mean Loss - Reconstruction",
                            {
                                'Train' : mlossX,
                                'Validation' : vmlossX
                            }, i+1)
                            
    tb_writer.flush()
    tb_writer.close()

In [9]:
# Prepare Data
data = pd.read_csv('TrainValid_Data.csv')
data['psi'] =  12*data['rho']/ (data['E']*data['t']**2)

num_freq_in = 4
num_freq_out = 10

features = freq_name(num_freq_out,1,0)
latent = ['psi', 'nu', 'a', 'b']

train_split = int(0.8*len(data))
valid_split = len(data)- train_split

scaled_data = data[features+latent].copy()

scaled_data['psi'] = np.log(scaled_data['psi']) 
scaled_data[freq_name(num_freq_out,1,0)] = np.log(scaled_data[freq_name(num_freq_out,1,0)])

scaled_data = GSAE_data(scaled_data, features, latent)
train_set, valid_set = to_data.random_split(scaled_data, [train_split, valid_split])

In [10]:
# Train Model
# Parameters
num_features = len(features)
num_latent = len(latent)
he_nodes = [10,50,100,200,300,400,300,200,100,50,10]
hd_nodes = [10,50,100,200,300,400,300,200,100,50,10]
hactiv = 'prelu'

batch_size_train = 2000
batch_size_valid = 2000

epochs = 200
learn_rate = 1e-3

# Optim Selections
loss_function = to.nn.SmoothL1Loss()
optimizer_type = 'adam'

# Data loaders
train_loader = to.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=True)
valid_loader = to.utils.data.DataLoader(valid_set, batch_size=batch_size_valid, shuffle=True)

# Model
model = GSAE_Network(num_freq_in, num_latent, num_freq_out, he_nodes, hd_nodes, hactiv)

In [11]:
# Train
train_GSAE(
    model, num_freq_in, num_freq_out,
    train_loader, valid_loader,
    loss_function, optimizer_type,
    epochs, learn_rate)

to.save(model.state_dict(), 'GSAE_model.state')

--------------------------------------------------
Epoch 1 / 200
---------------
Average Train Loss : 10.622230529785156
Average Validation Loss : 6.4720635414123535
--------------------------------------------------
Epoch 2 / 200
---------------
Average Train Loss : 4.804290771484375
Average Validation Loss : 3.064760208129883
--------------------------------------------------
Epoch 3 / 200
---------------
Average Train Loss : 1.6999393701553345
Average Validation Loss : 0.6723567843437195
--------------------------------------------------
Epoch 4 / 200
---------------
Average Train Loss : 0.3363741636276245
Average Validation Loss : 0.16747459769248962
--------------------------------------------------
Epoch 5 / 200
---------------
Average Train Loss : 0.17502914369106293
Average Validation Loss : 0.17657910287380219
--------------------------------------------------
Epoch 6 / 200
---------------
Average Train Loss : 0.1552240401506424
Average Validation Loss : 0.25859591364860535
--

In [14]:
model.load_state_dict(to.load('GSAE_model.state'))

test_data = pd.read_csv('Test_Data.csv')
test_data['psi'] =  12*test_data['rho']/ (test_data['E']*test_data['t']**2)

train_split = int(0.8*len(test_data))
valid_split = len(test_data)- train_split

scaled_test_data = test_data[features+latent].copy()

scaled_test_data['psi'] = np.log(scaled_test_data['psi'])
scaled_test_data[freq_name(num_freq_out,1,0)] = np.log(scaled_test_data[freq_name(num_freq_out,1,0)])

scaled_test_data = GSAE_data(scaled_test_data, features, latent)
test_loader = to.utils.data.DataLoader(scaled_test_data, batch_size=len(scaled_test_data), shuffle=False)

model.eval()
with to.no_grad():
    for i, data in enumerate(test_loader):
        X, Y = data
        predictY = model.encoder(X[:,0:num_freq_in])
        Y[:,0] = to.exp(Y[:,0])
        predictY[:,0] = to.exp(predictY[:,0])
        abs_perc_error = to.abs((Y - predictY)/Y)*100
        MAPE_per_dim = to.mean(abs_perc_error, 0)

        print('Absolute Percentage Errors: ')
        print('-'*20)
        print('D : {:0.2f} %'.format(MAPE_per_dim[0].item()))
        print('m_A : {:0.2f} %'.format(MAPE_per_dim[1].item()))
        print('rho : {:0.2f} %'.format(MAPE_per_dim[2].item()))
        print('a : {:0.2f} %'.format(MAPE_per_dim[3].item()))
        # print('b : {:0.2f} %'.format(MAPE_per_dim[4].item()))
        # print('t : {:0.2f} %'.format(MAPE_per_dim[5].item()))

        # plt.figure(figsize=(10,10))
        # plt.plot(['E', r'$\nu$', r'$\rho$', 'a', 'b', 't'], MAPE_per_dim.cpu(), 'b:o')
        # plt.ylim(0,100)
        # plt.grid(axis='y')
        # plt.yticks(np.arange(0,105,5))
        # plt.xticks(fontsize=16)
        # plt.title('GSAE MAPE Plot', fontsize=20)
        # plt.ylabel('MAPE [%]', fontsize=18)
        # plt.xlabel('Parameter', fontsize=18)
        # plt.savefig('GSAE_MAPE_plot.pdf', dpi=1200, bbox_inches='tight')
        # plt.show()

        np.savetxt('GSAE_MAPE_test.txt', MAPE_per_dim.cpu().numpy())

Absolute Percentage Errors: 
--------------------
D : 210.67 %
m_A : 32.16 %
rho : 25.66 %
a : 61.65 %
